In [99]:
import os
import pandas as pd
import numpy as np
import random
from prettytable import PrettyTable



In [103]:
# 指定路径
os.chdir('E:\Jupyterspace')

# 读取知识库
inputfile = 'test04.xlsx'
symptom = pd.read_excel(inputfile,dtype={'rate_id':str,'score':str})
symptom.head()

# 产生随机样例
# random.seed(1234)
sample = []
l = symptom['no']
k = random.randint(1,10)
n = random.sample(list(l), k) 
sample = pd.DataFrame(n,columns=['no'])

# 关联疾病、类型、等级
sample_ill = sample.merge(symptom)
sample_ill.drop_duplicates(subset=['symptom_attri_id'],keep='first',inplace=True)

# 患者症状list
symptom_list = sample_ill.loc[:,['symptom_id','symptom_attri_value']]
symptom_list = symptom_list.drop_duplicates()
symptom_list = symptom_list.sort_values(by = ['symptom_id'],axis = 0,ascending = 1)
symptom_list = symptom_list['symptom_attri_value']
train_data1 = np.array(symptom_list)#np.ndarray()
symptom_list=train_data1.tolist()#list


# num：按疾病、类型、等级分组，统计symptom_id去重数量
sympton_num = sample_ill.groupby(['relation_illn_id','type','class']).agg({'symptom_id': pd.Series.nunique})
sympton_num.columns = ['num']
sympton_num = sympton_num.reset_index()

# 提取关联疾病列表
ill_source = pd.DataFrame(sample_ill,columns=['relation_illn_id','relation_illn_name','relation_type','relation_type_attri_value',
                                            'type','class','rate_id'])

# 增加症状数量num
ill_source = ill_source.merge(sympton_num,on = ['relation_illn_id','type','class'])
# ill_source = ill_source.set_index('relation_illn_id')

# 排序
ill_rank = pd.DataFrame(ill_source,columns=['type','class','num','rate_id'])
train_data = np.array(ill_rank)  
out_list = train_data.tolist() 
rank_list = []
for i in out_list:
    str1 = ''.join([str(e) for e in i])
    rank_list.append(str1)
    
rank_list = pd.DataFrame(rank_list,columns=['sorc'])
rank_list

ill_rank =pd.concat([ill_source.reset_index(drop=True),rank_list],axis=1)


ill_rank = ill_rank.sort_values(by = ['sorc'],axis = 0,ascending = 0)

ill_rank = ill_rank.drop_duplicates('relation_illn_id')
ill_rank['rank']=ill_rank['sorc'].rank(ascending=0,method='dense')
ill_rank = ill_rank.sort_values(by = ['rank','relation_illn_id'],axis = 0,ascending = [True, False])

# 疑似疾病与排名
ill_list = ill_rank.loc[:,['rank','relation_illn_name']]
ill_list = ill_list.drop_duplicates()

train_data2 = np.array(ill_list)  
ill_list_list=train_data2.tolist() 
ill_list_list 

# 排序原因
# 排序原因
labels1 = ['relation_illn_name','symptom_attri_value','relation_type','relation_type_attri_value','rate',
          'relation_illn_id','type','class']
# 提取疾病、症状、关系类型、等级、症状权重
reason_list = sample_ill[labels]
# 关联疾病rank
reason_list =ill_list.merge(reason_list,on = ['relation_illn_name'])
reason_list = reason_list.sort_values(by = ['rank','relation_illn_id','type','class','rate'],
                                      ascending=[True, False, False, False, False])
labels2 = ['rank','relation_illn_name','symptom_attri_value','relation_type','relation_type_attri_value','rate']
reason_list = reason_list[labels2]

train_data3 = np.array(reason_list)  
reason_list=train_data3.tolist() 
x= PrettyTable(["排序","疑似疾病", "症状", "关系类型","关系类型属性值","症状权重"])
for i in reason_list:
    x.add_row(i)
    
# 将样例、疑似疾病排序、原因输出到文件
filename = 'sample_out.txt'
f = open (filename,'a')
print('患者症状如下：',file = f)
s3 = ";".join([str(e) for e in symptom_list])
print(s3,file = f)
print('疑似疾病如下：',file = f)
s4 = ";".join([str(e) for e in ill_list_list])
print(s4,file = f)
print('原因如下：',file = f)
print(x,file = f)
print("\n\n\n",file = f)
f.close()

In [111]:
for j in range(1,10):
    sample = []
    l = symptom['no']
    k = random.randint(1,10)
    n = random.sample(list(l), k) 
    sample = pd.DataFrame(n,columns=['no'])
    sample_ill = sample.merge(symptom)
    sample_ill.drop_duplicates(subset=['symptom_attri_id'],keep='first',inplace=True)
    symptom_list = sample_ill.loc[:,['symptom_id','symptom_attri_value']]
    symptom_list = symptom_list.drop_duplicates()
    symptom_list = symptom_list.sort_values(by = ['symptom_id'],axis = 0,ascending = 1)
    symptom_list = symptom_list['symptom_attri_value']
    train_data1 = np.array(symptom_list)#np.ndarray()
    symptom_list=train_data1.tolist()#list
    sympton_num = sample_ill.groupby(['relation_illn_id','type','class']).agg({'symptom_id': pd.Series.nunique})
    sympton_num.columns = ['num']
    sympton_num = sympton_num.reset_index()
    ill_source = pd.DataFrame(sample_ill,columns=['relation_illn_id','relation_illn_name','relation_type','relation_type_attri_value','type','class','rate_id'])
    ill_source = ill_source.merge(sympton_num,on = ['relation_illn_id','type','class'])
    ill_rank = pd.DataFrame(ill_source,columns=['type','class','num','rate_id'])
    train_data = np.array(ill_rank)  
    out_list = train_data.tolist() 
    rank_list = []
    for i in out_list:
        str1 = ''.join([str(e) for e in i])
        rank_list.append(str1)
    rank_list = pd.DataFrame(rank_list,columns=['sorc'])
    ill_rank =pd.concat([ill_source.reset_index(drop=True),rank_list],axis=1)
    ill_rank = ill_rank.sort_values(by = ['sorc'],axis = 0,ascending = 0)
    ill_rank = ill_rank.drop_duplicates('relation_illn_id')
    ill_rank['rank']=ill_rank['sorc'].rank(ascending=0,method='dense')
    ill_rank = ill_rank.sort_values(by = ['rank','relation_illn_id'],axis = 0,ascending = [True, False])
    ill_list = ill_rank.loc[:,['rank','relation_illn_name']]
    ill_list = ill_list.drop_duplicates()
    train_data2 = np.array(ill_list)  
    ill_list_list=train_data2.tolist() 
    ill_list_list 
    labels1 = ['relation_illn_name','symptom_attri_value','relation_type','relation_type_attri_value','rate','relation_illn_id','type','class']
    reason_list = sample_ill[labels]
    reason_list =ill_list.merge(reason_list,on = ['relation_illn_name'])
    reason_list = reason_list.sort_values(by = ['rank','relation_illn_id','type','class','rate'],ascending=[True, False, False, False, False])
    labels2 = ['rank','relation_illn_name','symptom_attri_value','relation_type','relation_type_attri_value','rate']
    reason_list = reason_list[labels2]
    train_data3 = np.array(reason_list)  
    reason_list=train_data3.tolist() 
    x= PrettyTable(["排序","疑似疾病", "症状", "关系类型","关系类型属性值","症状权重"])
    for i in reason_list:x.add_row(i)
    filename = 'sample_out.txt'
    f = open (filename,'a')
    print('测试样例：%s' %j ,file = f)
    print('患者症状如下：',file = f)
    s3 = ";".join([str(e) for e in symptom_list])
    print(s3,file = f)
    print('疑似疾病如下：',file = f)
    s4 = ";".join([str(e) for e in ill_list_list])
    print(s4,file = f)
    print('原因如下：',file = f)
    print(x,file = f)
    print("\n\n\n",file = f)
    f.close()

    
    